[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/deployment.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239303-lesson-8-deployment)

# Деплой (Deployment)

## Итоги предыдущего шага

Мы создали агента с памятью:

* `act` - позволять модели вызывать определенные инструменты
* `observe` - передавать результат работы инструмента обратно модели
* `reason` - позволять модели анализировать результат инструмента для принятия решения о следующих действиях (например, вызвать другой инструмент или ответить напрямую)
* `persist state` - использовать механизм контрольных точек в памяти для поддержки длительных диалогов с перерывами

## Цели текущего раздела

Теперь мы рассмотрим, как развернуть нашего агента локально в Studio и в `LangGraph Cloud`.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph_sdk langchain_core

## Основные понятия

Есть несколько ключевых концепций, которые важно понимать:

`LangGraph` —
- Библиотека для Python и JavaScript
- Позволяет создавать агентские workflow

`LangGraph API` —
- Объединяет код графа
- Предоставляет очередь задач для управления асинхронными операциями
- Обеспечивает персистентность для сохранения состояния между взаимодействиями

`LangSmith Deployment` (ранее `LangGraph Cloud`) —
- Хостинг-сервис для LangGraph API
- Позволяет развертывать графы из GitHub-репозиториев
- Также предоставляет мониторинг и трассировку для развернутых графов
- Доступен через уникальный URL для каждого развертывания

`LangSmith Studio` (ранее `LangGraph Studio`) —
- Интегрированная среда разработки (IDE) для LangGraph-приложений
- Использует API в качестве бэкенда, позволяя тестировать и исследовать графы в реальном времени
- Может запускаться локально или с облачным развертыванием (см. ниже)

`LangGraph SDK` —
- Python-библиотека для программного взаимодействия с LangGraph-графами
- Предоставляет единый интерфейс для работы с графами, независимо от того, развернуты они локально или в облаке
- Позволяет создавать клиентов, получать доступ к ассистентам, управлять потоками и выполнять запуски

## Локальное тестирование

## Studio

**⚠️ Внимание**

После создания этих видео мы обновили Studio, и теперь её можно запускать локально и открывать в браузере. Это предпочтительный способ работы со Studio вместо использования Desktop App, показанного в видео. Теперь она называется _LangSmith Studio_ вместо _LangGraph Studio_. Подробные инструкции по настройке доступны в руководстве "Начало работы" в начале курса. Описание Studio можно найти [здесь](https://docs.langchain.com/langsmith/studio), а конкретные детали по локальному развертыванию — [здесь](https://docs.langchain.com/langsmith/quick-start-studio#local-development-server).

Чтобы запустить локальный сервер разработки, выполните следующую команду в терминале в директории `/studio` этого модуля:

```
langgraph dev
```

В ответ вы должны увидеть следующий вывод:
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

Откройте браузер и перейдите по адресу Studio UI, указанному выше.

In [2]:
if 'google.colab' in str(get_ipython()):
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab")

In [3]:
from langgraph_sdk import get_client

In [6]:
# URL локального сервера разработки
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

# Поиск всех размещенных графов
assistants = await client.assistants.search()

In [5]:
assistants[-3]

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'context': {},
 'metadata': {'created_by': 'system'},
 'name': 'agent',
 'created_at': '2025-11-02T10:08:44.148230+00:00',
 'updated_at': '2025-11-02T10:08:44.148230+00:00',
 'version': 1,
 'description': None}

In [7]:
# Создаем поток для отслеживания состояния запуска графа
thread = await client.threads.create()

Теперь мы можем запустить нашего агента [с помощью `client.runs.stream`](https://docs.langchain.com/oss/python/langgraph/graph-api/#stream-and-astream), указав:

* `thread_id`
* `graph_id`
* `input`
* `stream_mode`

Мы подробно обсудим потоковую передачу в следующем модуле.

Пока просто учтите, что мы используем [потоковую передачу](https://docs.langchain.com/langsmith/streaming) полного значения состояния после каждого шага графа с `stream_mode="values"`.

Состояние сохраняется в `chunk.data`.

In [8]:
from langchain_core.messages import HumanMessage

# Ввод
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Поток
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '2959c724-f431-47b6-bf15-ee7cc7c082b5'}
{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'qV69MRwP9', 'function': {'name': 'multiply', 'arguments': '{"a": 3, "b": 2}'}, 'index': 0}]}, 'response_metadata': {'token_usage': {'prompt_tokens': 274, 'total_tokens': 291, 'completion_tokens': 17}, 'model_name': 'mistral-small-latest', 'model': 'mistral-small-latest', 'finish_reason': 'tool_calls', 'model_provider': 'mistralai'}, 'type': 'ai', 'name': None, 'id': 'lc_run--55af4c59-9aba-464d-99bc-cae9b5b1fb14-0', 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'qV69MRwP9', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 274, 'output_tokens': 17, 'total_tokens': 291}}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': 'd6b3abad-1ecb-41ad-9690-024253587c39',

### Подключение LangSmith к вашему GitHub-репозиторию

* Перейдите в [LangSmith](https://smith.langchain.com/)
* Нажмите на вкладку `deployments` в левой панели LangSmith
* Добавьте `+ New Deployment`
* Затем выберите GitHub-репозиторий (например, `langchain-academy`), который вы только что создали для курса
* Укажите `LangGraph API config file` на одну из директорий `studio`
* Например, для module-1 используйте: `module-1/studio/langgraph.json`
* Установите ваши API-ключи (например, можно просто скопировать из вашего файла `module-1/studio/.env`)

![Screenshot 2024-09-03 at 11.35.12 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### Работа с вашим развертыванием

Затем мы можем взаимодействовать с нашим развертыванием несколькими способами:

* С помощью SDK, как и раньше
* Через [LangGraph Studio](https://docs.langchain.com/langsmith/deployment-quickstart#3-test-your-application-in-studio)

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

Чтобы использовать SDK здесь, в блокноте, просто убедитесь, что `LANGSMITH_API_KEY` установлен!

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGSMITH_API_KEY")

In [ ]:
# Замените это на URL вашего развернутого графа
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# Найти все размещенные графы
assistants = await client.assistants.search()

In [37]:
# Выберите агента
agent = assistants[0]

In [38]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [40]:
from langchain_core.messages import HumanMessage

# Мы создаём поток (thread) для отслеживания состояния нашего запуска
thread = await client.threads.create()

# Ввод
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Поток
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c